<a href="https://colab.research.google.com/github/AYUSH-11/Machine-Learning/blob/main/Movies_Recommended_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#All mathematical operation are done by numpy
import numpy as np
#It is used for ploting 
import matplotlib.pyplot as plt
# for read all files
import pandas as pd
import math
import random
from google.colab import drive  #to import Google drive data
drive.mount('/content/drive') #to the drive data into colab cloud and enter the activation colab 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/"My Drive/Colab Notebooks/Linear Regression"

/content/drive/My Drive/Colab Notebooks/Linear Regression


In [3]:
dataset_1 = pd.read_csv('rating.csv')
dataset_2 = pd.read_csv('titles.csv')

In [4]:
dataset_1.head()

,user_id,movie_id,rating,timestamp
0,172,94969,5.0,1396067836
1,172,98956,4.0,1396067879
2,176,73881,4.0,1499807147
3,221,1900,4.5,1288550866
4,333,33688,4.0,1412015122


In [5]:
#Drop removes a specified row or column from a dataframe
dataset_1 = dataset_1.drop('timestamp', 1)

In [6]:
dataset_1.head()

,user_id,movie_id,rating
0,172,94969,5.0
1,172,98956,4.0
2,176,73881,4.0
3,221,1900,4.5
4,333,33688,4.0


In [7]:
dataset_2.head()

,movie_id,title
0,139,Target (1995)
1,568,Bhaji on the Beach (1993)
2,1900,Children of Heaven The (Bacheha-Ye Aseman) (1...
3,2284,Bandit Queen (1994)
4,5985,Asoka (Ashoka the Great) (2001)


In [8]:
"""Remove Year from title and add into separate column"""
dataset_2['year'] = dataset_2.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
dataset_2['year'] = dataset_2.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
dataset_2['title'] = dataset_2.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
dataset_2['title'] = dataset_2['title'].apply(lambda x: x.strip())

In [9]:
dataset_2.head()

,movie_id,title,year
0,139,Target,1995
1,568,Bhaji on the Beach,1993
2,1900,Children of Heaven The (Bacheha-Ye Aseman),1997
3,2284,Bandit Queen,1994
4,5985,Asoka (Ashoka the Great),2001


In [10]:
#Remove genre
#dataset_2 = dataset_2.drop('genres', 1)

In [11]:
dataset_2.head()

,movie_id,title,year
0,139,Target,1995
1,568,Bhaji on the Beach,1993
2,1900,Children of Heaven The (Bacheha-Ye Aseman),1997
3,2284,Bandit Queen,1994
4,5985,Asoka (Ashoka the Great),2001


In [12]:
userInput = [
            {'title':'3 Idiots', 'rating':5},
            {'title':'Satyameva Jayate', 'rating':3.5},
            {'title':'Fan', 'rating':1},
            {'title':"M.S. Dhoni: The Untold Story", 'rating':5},
            {'title':'Kaabil', 'rating':4.5},
            {'title':'Raees', 'rating':4.5}
             
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,3 Idiots,5.0
1,Satyameva Jayate,3.5
2,Fan,1.0
3,M.S. Dhoni: The Untold Story,5.0
4,Kaabil,4.5
5,Raees,4.5


In [13]:
#Filtering out the movies by title
inputId = dataset_2[dataset_2['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

In [14]:
inputMovies

,movie_id,title,rating
0,73881,3 Idiots,5.0
1,157106,Fan,1.0
2,169888,Kaabil,4.5
3,183979,M.S. Dhoni: The Untold Story,5.0
4,184193,Raees,4.5
5,193139,Satyameva Jayate,3.5


In [15]:
#Now we can select people who has watch input movies or give rating to input movies
userSubset = dataset_1[dataset_1['movie_id'].isin(inputMovies['movie_id'].tolist())]
userSubset.head()

,user_id,movie_id,rating
2,176,73881,4.0
10,422,73881,5.0
12,505,73881,4.0
13,516,73881,4.0
25,745,73881,3.0


In [16]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
#Group with same user id
userSubsetGroup = userSubset.groupby(['user_id'])

In [17]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [18]:
userSubset

,user_id,movie_id,rating
2,176,73881,4.0
10,422,73881,5.0
12,505,73881,4.0
13,516,73881,4.0
25,745,73881,3.0
...,...,...,...
17515,282309,73881,5.0
17534,282717,73881,4.5
17551,282853,73881,4.5
17591,283000,73881,4.0


In [19]:
userSubsetGroup = userSubsetGroup[0:100]

In [20]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movie_id')
    inputMovies = inputMovies.sort_values(by='movie_id')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/math.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [21]:
pearsonCorrelationDict.items()

dict_items([(21962, 0.0), (41110, 0.596039560679274), (68886, 0.9971764649527379), (184787, 0.0), (190944, 0.11470786693528087), (210634, 0.7559289460184535), (223644, -0.917662935482247), (5544, 1.0), (6080, 1.0), (23357, 0), (30929, 1.0), (33766, 1.0), (38455, 1.0), (53263, 1.0), (57369, -1.0), (64677, 1.0), (65868, 1.0), (70853, 1.0), (87062, 1.0), (91292, 1.0), (123100, 1.0), (131584, -1.0), (133425, -1.0), (138487, 1.0), (148948, 0), (153584, 1.0), (165612, 0), (182942, 1.0), (195998, 1.0), (205115, 1.0), (226641, 1.0), (260396, 1.0), (263047, 0), (271113, 1.0), (176, 0), (422, 0), (505, 0), (516, 0), (745, 0), (773, 0), (1147, 0), (1407, 0), (1507, 0), (1593, 0), (1742, 0), (1867, 0), (1980, 0), (2069, 0), (2098, 0), (2168, 0), (2184, 0), (2358, 0), (2968, 0), (3203, 0), (3450, 0), (3481, 0), (3770, 0), (4109, 0), (4192, 0), (4203, 0), (4226, 0), (4544, 0), (4559, 0), (4646, 0), (4692, 0), (4794, 0), (5012, 0), (5446, 0), (5747, 0), (6337, 0), (6766, 0), (6834, 0), (6877, 0), (71

In [22]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.000000,21962
1,0.596040,41110
2,0.997176,68886
3,0.000000,184787
4,0.114708,190944


In [23]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
31,1.0,260396
13,1.0,53263
29,1.0,205115
28,1.0,195998
27,1.0,182942


In [24]:
dataset_1.head()

,user_id,movie_id,rating
0,172,94969,5.0
1,172,98956,4.0
2,176,73881,4.0
3,221,1900,4.5
4,333,33688,4.0


In [25]:
topUsersRating=topUsers.merge(dataset_1, left_on='userId', right_on='user_id', how='inner')
topUsersRating.head()

,similarityIndex,userId,user_id,movie_id,rating
0,1.0,260396,260396,5985,3.5
1,1.0,260396,260396,27131,3.5
2,1.0,260396,260396,27163,3.0
3,1.0,260396,260396,34435,4.5
4,1.0,260396,260396,34466,2.5


In [26]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,user_id,movie_id,rating,weightedRating
0,1.0,260396,260396,5985,3.5,3.5
1,1.0,260396,260396,27131,3.5,3.5
2,1.0,260396,260396,27163,3.0,3.0
3,1.0,260396,260396,34435,4.5,4.5
4,1.0,260396,260396,34466,2.5,2.5


In [27]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movie_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movie_id,,
568,1.000000,2.500000
1900,2.000000,8.500000
2284,1.000000,5.000000
5985,2.114708,6.401478
6244,1.000000,4.500000


In [28]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movie_id,,
568,2.500000,568
1900,4.250000,1900
2284,5.000000,2284
5985,3.027121,5985
6244,4.500000,6244


In [29]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movie_id,,
168168,5.000000,168168
2284,5.000000,2284
146366,5.000000,146366
135589,5.000000,135589
137490,5.000000,137490
152011,4.864393,152011
149606,4.676730,149606
135204,4.637146,135204
142382,4.588562,142382


In [30]:
dataset_2.loc[dataset_2['movie_id'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movie_id,title,year
3,2284,Bandit Queen,1994
229,135204,Madras Cafe,2013
236,135589,1920,2008
277,137490,Three Daughters (Teen Kanya),1961
297,142382,Agneepath,2012
387,146366,Kaun,1999
418,149606,Bajirao Mastani,2015
445,152011,Anjaana Anjaani,2010
575,154348,Jolly LLB,2013
667,168168,Margarita with a Straw,2014
